In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split


Note: I am dropping medication as it is using too much memory and has too many unique drug names. So it is not feasible for merging

In [2]:
# Load datasets
admissionDX = pd.read_csv("admissionDx.csv")
patient = pd.read_csv("patient.csv")
lab = pd.read_csv("lab.csv")
diagnosis = pd.read_csv("diagnosis.csv")
#medication = pd.read_csv("C:/Users/Natasha/Desktop/Data_Exploration_2/medication.csv")
treatment = pd.read_csv("treatment.csv")


C:\Users\Natasha\AppData\Local\Temp\ipykernel_252\2040081847.py:6: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  medication = pd.read_csv("C:/Users/Natasha/Desktop/Data_Exploration_2/medication.csv")


In [3]:
# Check the structure and size of each dataset
print(admissionDX.info())
print(patient.info())
print(lab.info())
print(diagnosis.info())
#print(medication.info())
print(treatment.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7578 entries, 0 to 7577
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   admissiondxid         7578 non-null   int64 
 1   patientunitstayid     7578 non-null   int64 
 2   admitdxenteredoffset  7578 non-null   int64 
 3   admitdxpath           7578 non-null   object
 4   admitdxname           7578 non-null   object
 5   admitdxtext           7578 non-null   object
dtypes: int64(3), object(3)
memory usage: 355.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2520 entries, 0 to 2519
Data columns (total 29 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   patientunitstayid          2520 non-null   int64  
 1   patienthealthsystemstayid  2520 non-null   int64  
 2   gender                     2516 non-null   object 
 3   age                        2516 non-null   ob

In [4]:
# Keep only relevant columns for each dataset

# admissionDX
admissionDX = admissionDX[['patientunitstayid', 'admitdxname', 'admitdxtext']]

# patient
patient = patient[['patientunitstayid', 'age', 'gender', 'ethnicity', 
                   'hospitaladmitsource', 'hospitaldischargelocation', 
                   'hospitaldischargestatus']]

# lab
lab = lab[['patientunitstayid', 'labname', 'labresult']]

# diagnosis
diagnosis = diagnosis[['patientunitstayid', 'diagnosisstring', 'icd9code']]

# medication
#medication = medication[['patientunitstayid', 'drugname', 'dosage', 
#                         'routeadmin', 'drugstartoffset', 'drugstopoffset']]

# treatment
treatment = treatment[['patientunitstayid', 'treatmentstring']]

# Display the first few rows of each dataset to confirm changes
print("admissionDX columns:", admissionDX.columns)
print("patient columns:", patient.columns)
print("lab columns:", lab.columns)
print("diagnosis columns:", diagnosis.columns)
print("medication columns:", medication.columns)
print("treatment columns:", treatment.columns)


admissionDX columns: Index(['patientunitstayid', 'admitdxname', 'admitdxtext'], dtype='object')
patient columns: Index(['patientunitstayid', 'age', 'gender', 'ethnicity',
       'hospitaladmitsource', 'hospitaldischargelocation',
       'hospitaldischargestatus'],
      dtype='object')
lab columns: Index(['patientunitstayid', 'labname', 'labresult'], dtype='object')
diagnosis columns: Index(['patientunitstayid', 'diagnosisstring', 'icd9code'], dtype='object')
medication columns: Index(['patientunitstayid', 'drugname', 'dosage', 'routeadmin',
       'drugstartoffset', 'drugstopoffset'],
      dtype='object')
treatment columns: Index(['patientunitstayid', 'treatmentstring'], dtype='object')


In [5]:
#drop duplicates
admissionDX.drop_duplicates(inplace=True)
lab.drop_duplicates(inplace=True)
diagnosis.drop_duplicates(inplace=True)
#medication.drop_duplicates(inplace=True)
treatment.drop_duplicates(inplace=True)


In [6]:
# Filter patient IDs for a subset
subset_patient_ids = patient['patientunitstayid'].sample(n=1000, random_state=42)

# Filter each dataset by these IDs
admissionDX = admissionDX[admissionDX['patientunitstayid'].isin(subset_patient_ids)]
lab = lab[lab['patientunitstayid'].isin(subset_patient_ids)]
diagnosis = diagnosis[diagnosis['patientunitstayid'].isin(subset_patient_ids)]
#medication = medication[medication['patientunitstayid'].isin(subset_patient_ids)]
treatment = treatment[treatment['patientunitstayid'].isin(subset_patient_ids)]


In [ ]:
# Check the structure and size of each dataset
print(admissionDX.info())
print(patient.info())
print(lab.info())
print(diagnosis.info())
#print(medication.info())
print(treatment.info())

In [7]:
# AdmissionDX: No changes required as types seem correct
admissionDX['patientunitstayid'] = admissionDX['patientunitstayid'].astype("int32")

# Patient
# Convert 'age' to numeric, setting invalid parsing as NaN, and use Int32 for nullable integer
patient['age'] = pd.to_numeric(patient['age'], errors='coerce').astype("float32")
patient['patientunitstayid'] = patient['patientunitstayid'].astype("int32")

# Lab
lab['patientunitstayid'] = lab['patientunitstayid'].astype("int32")
lab['labresult'] = lab['labresult'].astype("float32")  # Use float32 for less memory usage

# Diagnosis
diagnosis['patientunitstayid'] = diagnosis['patientunitstayid'].astype("int32")
diagnosis['icd9code'] = diagnosis['icd9code'].astype("category")  # Use category for 'icd9code'

"""
Medication
medication['dosage'] = pd.to_numeric(medication['dosage'], errors='coerce').astype("float32")
medication['patientunitstayid'] = medication['patientunitstayid'].astype("int32")
medication['drugstartoffset'] = medication['drugstartoffset'].astype("int32")
medication['drugstopoffset'] = medication['drugstopoffset'].astype("int32")
"""

# Treatment
treatment['patientunitstayid'] = treatment['patientunitstayid'].astype("int32")
treatment['treatmentstring'] = treatment['treatmentstring'].astype("category")  # Use category for 'treatmentstring'

# Convert object columns to 'category' type where applicable to save memory
patient['gender'] = patient['gender'].astype("category")
patient['ethnicity'] = patient['ethnicity'].astype("category")
patient['hospitaladmitsource'] = patient['hospitaladmitsource'].astype("category")
patient['hospitaldischargelocation'] = patient['hospitaldischargelocation'].astype("category")
patient['hospitaldischargestatus'] = patient['hospitaldischargestatus'].astype("category")

# Confirm data types after conversion
print("AdmissionDX dtypes:\n", admissionDX.dtypes)
print("Patient dtypes:\n", patient.dtypes)
print("Lab dtypes:\n", lab.dtypes)
print("Diagnosis dtypes:\n", diagnosis.dtypes)
#print("Medication dtypes:\n", medication.dtypes)
print("Treatment dtypes:\n", treatment.dtypes)


AdmissionDX dtypes:
 patientunitstayid     int32
admitdxname          object
admitdxtext          object
dtype: object
Patient dtypes:
 patientunitstayid               int32
age                           float32
gender                       category
ethnicity                    category
hospitaladmitsource          category
hospitaldischargelocation    category
hospitaldischargestatus      category
dtype: object
Lab dtypes:
 patientunitstayid      int32
labname               object
labresult            float32
dtype: object
Diagnosis dtypes:
 patientunitstayid       int32
diagnosisstring        object
icd9code             category
dtype: object
Medication dtypes:
 patientunitstayid      int32
drugname              object
dosage               float32
routeadmin            object
drugstartoffset        int32
drugstopoffset         int32
dtype: object
Treatment dtypes:
 patientunitstayid       int32
treatmentstring      category
dtype: object


In [1]:
#print(merged_df.memory_usage(deep=True).sum() / (1024**2), "MB")
#print(medication.memory_usage(deep=True).sum() / (1024**2), "MB")


In [8]:
#merging
# Start with the base dataframe (AdmissionDX)
merged_df = admissionDX


In [9]:

# Merge with Patient data
merged_df = merged_df.merge(patient, on="patientunitstayid", how="left")


In [10]:

# Merge with Lab data
merged_df = merged_df.merge(lab, on="patientunitstayid", how="left")


In [11]:

# Merge with Diagnosis data
merged_df = merged_df.merge(diagnosis, on="patientunitstayid", how="left")


In [12]:
# Merge with Treatment data
merged_df = merged_df.merge(treatment, on="patientunitstayid", how="left")

In [13]:

# Random sampling to reduce dataset size
subset = merged_df.sample(n=1000, random_state=42)



In [14]:
# Save the smaller dataset
subset.to_csv('preprocessed.csv', index=False)


In [15]:
# Check the final dataframe structure
print(subset.info())


<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 10343652 to 2537717
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   patientunitstayid          1000 non-null   int32   
 1   admitdxname                1000 non-null   object  
 2   admitdxtext                1000 non-null   object  
 3   age                        976 non-null    float32 
 4   gender                     1000 non-null   category
 5   ethnicity                  997 non-null    category
 6   hospitaladmitsource        748 non-null    category
 7   hospitaldischargelocation  979 non-null    category
 8   hospitaldischargestatus    979 non-null    category
 9   labname                    1000 non-null   object  
 10  labresult                  997 non-null    float32 
 11  diagnosisstring            998 non-null    object  
 12  icd9code                   836 non-null    category
 13  treatmentstring            9